In [1]:
import torch
import torch.nn as nn
from torch._C import device
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import torch.optim as optim
import time
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset
plt.style.use(["science", "notebook", "no-latex"])

# sample data from HotQCD

In [2]:
df1 = pd.read_csv("./data/hotqcd_1407.6387_err.csv")
for i in range(1000):
    print("sample:%s"%i)
    s_over_T3= df1["s/T^3"] 
    sig_s = np.random.randn(50) * df1["ds"] 
    s_sample = s_over_T3 + sig_s

    D_over_T4 = df1["TraceA"] 
    sig_D = df1["dTraceA"] * np.random.randn(50) 
    D_sample = D_over_T4 + sig_D


    P_sample = (s_sample - D_sample)/4
    E_sample = (3 * s_sample + D_sample)/4
    plt.plot(df1["T"],D_sample, "r",alpha=1 - i/1000)
    plt.plot(df1["T"],s_sample, "b",alpha=1 - i/1000)

    np.savez("sample_%s"%i, T = df1["T"], s_over_T3 = s_sample, D_over_T4 = D_sample, P_over_T4 = P_sample, E_over_T4 = E_sample, sig_s = sig_s, sig_D = sig_D)


In [4]:
def plot_Delta(epoch, path, Tarr, DeltaPred, TraceAnomaly, Tc=0.155):
    if epoch % 500 != 0: return
    DeltaPred = DeltaPred.cpu().detach().numpy()
    TraceAnomaly = TraceAnomaly.cpu().detach().numpy()
    plt.plot(Tarr/Tc, DeltaPred.flatten()/Tarr**4, 'r-', label="network")
    plt.plot(Tarr/Tc, TraceAnomaly.flatten()/Tarr**4, 'b--', label="input")
    plt.legend(loc='best')
    plt.xlabel(r"$T/T_c$",loc="center")
    plt.ylabel(r"$(\varepsilon - 3 P)/T^4$",loc="center")
    plt.savefig( path + "Delta_vs_T_epoch%s.jpg"%epoch)
    plt.close()
    
def plot_ed(epoch, path, Tarr, EdPred, Earr, Tc=0.155):
    """
    inupt: epoch
    return: T as a function of enrgy density
    """
    if epoch % 500 != 0: return
    EdPred = EdPred.cpu().detach().numpy()
    plt.plot(Tarr/Tc, EdPred.flatten(), 'r-', label='network')
    plt.plot(Tarr/Tc, Earr.cpu().detach().numpy(), 'b--', label='input')
    plt.xlabel(r"$T\ {\rm [GeV]}$",loc="center")
    plt.ylabel(r"$\varepsilon\ {\rm [GeV]^{4}}$",loc="center")
    plt.legend(loc='best')
    plt.savefig(path + "Ed_vs_T_epoch%s.jpg"%epoch)
    plt.close()
    
def plot_pr(epoch, path, Tarr, PrPred, Parr, Tc=0.155):
    """
    inupt: epoch
    return: T as a function of Pressure
    """
    if epoch % 500 != 0: return
    PrPred = PrPred.cpu().detach().numpy()
    plt.plot(Tarr/Tc, PrPred.flatten(), 'r-', label='network')
    plt.plot(Tarr/Tc, Parr.cpu().detach().numpy(), 'b--', label='input')
    plt.xlabel(r"$T\ {\rm [GeV]}$",loc="center")
    plt.ylabel(r"$P\ {\rm [GeV]^{4}}$",loc="center")
    plt.legend(loc='best')
    plt.savefig(path + "Pr_vs_T_epoch%s.jpg"%epoch)
    plt.close()

def plot_entropy(epoch, path, Tarr, SPred, Sarr, Tc=0.155):
    """
    inupt: epoch
    return: T as a function of entropy
    """
    if epoch % 500 != 0: return
    SPred = SPred.cpu().detach().numpy()
    plt.plot(Tarr/Tc, SPred.flatten(), 'r-', label='network')
    plt.plot(Tarr/Tc, Sarr.cpu().detach().numpy(), 'b--', label='input')
    plt.xlabel(r"$T\ {\rm [GeV]}$",loc="center")
    plt.ylabel(r"$S\ {\rm [GeV]^{3}}$",loc="center")
    plt.legend(loc='best')
    plt.savefig(path + "S_vs_T_epoch%s.jpg"%epoch)
    plt.close()
     

In [5]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, channels):
        super(ResidualBlock,self).__init__()
        self.channels = channels
        
        self.l1 = nn.Linear(channels,channels)
        self.l2 = nn.Linear(channels,channels)
    
    def forward(self, x):
        y = torch.sigmoid(self.l1(x))
        y = self.l2(y)
        return x+y
    
class Net_Mass(nn.Module):
    def __init__(self, input,  output):
        """
        input: tensor and shape (None, 1)
        NL: the number of layers
        NN: the number of neurons
        activate function: sigmoid
        output: tensor and shape (None, 1)
        """
        super(Net_Mass,self).__init__()
        self.input_layer1 = nn.Linear(input, 16)
        self.hidden_layers0= nn.Linear(16, 32)
        self.hidden_layers1 = ResidualBlock(32)
        self.hidden_layers2 = ResidualBlock(32)
        self.hidden_layers3 = ResidualBlock(32)
        self.hidden_layers4 = ResidualBlock(32)
        self.hidden_layers5 = ResidualBlock(32)
        self.hidden_layers6 = ResidualBlock(32)
        self.hidden_layers7 = ResidualBlock(32)
        self.hidden_layers8 = nn.Linear(32,16)
        self.output_layer = nn.Linear(16,output)
        
    def forward(self,x):
        
        o = self.act(self.input_layer1(x))
        o = self.act(self.hidden_layers0(o))
        o = self.act(self.hidden_layers1(o))
        o = self.act(self.hidden_layers2(o))
        o = self.act(self.hidden_layers3(o))
        o = self.act(self.hidden_layers4(o))
        o = self.act(self.hidden_layers5(o))
        o = self.act(self.hidden_layers6(o))
        o = self.act(self.hidden_layers7(o))
        o = self.act(self.hidden_layers8(o))

            
        opt = self.output_layer(o)
        opt = self.act1(opt)
        return opt
    
    def act(self,x):
        return x * torch.sigmoid(x)
    
    def act1(self,x):
        return 3 * torch.sigmoid(x)
    

In [6]:
def lnZ_qg(T, m, eta):
    deg = 50
    xk, wk = np.polynomial.laguerre.laggauss(deg=deg)

    pnodes = torch.from_numpy(xk).to(device)
    wnodes = torch.from_numpy(wk).to(device)
    rnodes =  torch.exp(-pnodes)
    psqure = pnodes**2
    
    E = (psqure + m ** 2) ** 0.5
    efactor = torch.exp(-E/T)
    f = efactor * eta
    f = f + 1
    f = torch.log(f)
    f = psqure * eta * f
    f = wnodes * f
    f = f/rnodes
    f = torch.sum(f,1)
    return f.reshape(-1,1)

In [7]:
def chi2(pre,obs,sig): 
    out = (obs - pre)**2/sig**2
    out = out.mean()
    return out

In [8]:
def Mass_T_train(learning_rate, epochs,  path_model, path_pic, path_data, device, run_seed):
    """
    return: loss, and save model
    """

    Net1 = Net_Mass(1,1).to(device)
    Net2 = Net_Mass(1,1).to(device)
    Net3 = Net_Mass(1,1).to(device)
    
    opt1 = optim.Adam(Net1.parameters(),lr = learning_rate)
    opt2 = optim.Adam(Net2.parameters(),lr = learning_rate)
    opt3 = optim.Adam(Net3.parameters(),lr = learning_rate)
    
    lr_step_size =2000

    schedular1 = lr_scheduler.StepLR(opt1, step_size=lr_step_size, gamma=0.9)    
    schedular2 = lr_scheduler.StepLR(opt2, step_size=lr_step_size, gamma=0.9)    
    schedular3 = lr_scheduler.StepLR(opt3, step_size=lr_step_size, gamma=0.9)    
    
    def init_normal(m):
        if type(m) == nn.Linear:
            nn.init.kaiming_uniform_(m.weight)
            
    Net1.apply(init_normal)
    Net1.train()
    Net2.apply(init_normal)
    Net2.train()    
    Net3.apply(init_normal)
    Net3.train()    
    

    
    ########## loss
    def loss_a(T,):

        T.requires_grad = True
        inp = T
        
        # Obtain Mass
        Mud = Net1(inp)
        Ms = Net2(inp)
        Mgluon = Net3(inp)
        

        coef =  1 / (2 * torch.pi**2)   
        ud_dof = 2 *  2 * 2 * 3  #u,d * q, qbar * spin_dof * color_dof
        s_dof = 2 * 2 * 3
        gluon_dof = 8 * 2       # color dof * polarization dof       
        
        lnZ_ud =  ud_dof * coef * lnZ_qg(T, Mud, 1) 
        lnZ_s = s_dof * coef * lnZ_qg(T, Ms,1)
        lnZ_gluon = gluon_dof * coef * lnZ_qg(T, Mgluon, -1)
        


        
        lnZ_tot = lnZ_ud + lnZ_s + lnZ_gluon
        dlnZdT =  torch.autograd.grad(lnZ_tot, T, grad_outputs=torch.ones_like(T).to(device),create_graph=True)[0]

        pr = T * lnZ_tot  
        ed = T * T * dlnZdT
        s_pred = (pr +  ed) / T
        Delta = (ed - 3 * pr)
        
        mass_loss_1 = torch.where(T >(2.5 * 0.150), torch.abs(Mgluon/Mud - 1.5), torch.tensor([0.]).to(device)) # HTL mass constraint
        mass_loss_2 = torch.where(T >(2.5 * 0.150), torch.abs((Ms  - Mud)/0.09 - 1), torch.tensor([0.]).to(device))
        L_mass =  (mass_loss_1 + mass_loss_2)


        # build the loss function
        loss_mae = nn.L1Loss()

        loss1 = chi2(s_true, s_pred, sigma_s).to(device)
        loss2 = chi2(D_true, Delta, sigma_D).to(device)
        
        loss_mc = loss_mae(L_mass , torch.zeros_like(L_mass)).to(device)

        loss_tot = loss1 + loss2 + loss_mc
      
        return loss_tot,  loss1, loss2, ed, pr, s_pred, Delta

    tic = time.time()

    l1= []
    l2= []
    l3= []
    Loss_mean = 1e5

    sam = np.load(path_data + "sample_%s.npz"%run_seed)
    LQCD = pd.read_csv("./data/hotqcd_1407.6387_err.csv")

    s_true = sam["s_over_T3"] * sam["T"] ** 3
    E_true = sam["E_over_T4"] * sam["T"] ** 4
    P_true = sam["P_over_T4"] * sam["T"] ** 4
    D_true = E_true - 3 * P_true
    sigma_s = LQCD["ds"] * LQCD["T"] ** 3
    sigma_D = LQCD["dTraceA"] * LQCD["T"] ** 4
    
    Tem = LQCD["T"]

        
    Tem = torch.FloatTensor(Tem).reshape(-1,1).to(device)     
    D_true = torch.FloatTensor(D_true).reshape(-1,1).to(device)     
    s_true = torch.FloatTensor(s_true).reshape(-1,1).to(device)     
    E_true = torch.FloatTensor(E_true).reshape(-1,1).to(device)     
    P_true = torch.FloatTensor(P_true).reshape(-1,1).to(device)     
    sigma_s  = torch.FloatTensor(sigma_s).reshape(-1,1).to(device)     
    sigma_D = torch.FloatTensor(sigma_D).reshape(-1,1).to(device)     


    

    # Each step of the training process:
    for epoch in range(epochs):

        Net1.zero_grad()        
        Net2.zero_grad()        
        Net3.zero_grad()     
        
        loss, loss1, loss2, energy, pre, entropy, Trace= loss_a(Tem)
        
        
        plot_entropy(epoch, path_pic,LQCD["T"],entropy, s_true)
        plot_Delta(epoch, path_pic,LQCD["T"],Trace,D_true)
        plot_pr(epoch, path_pic,LQCD["T"],pre, P_true)
        plot_ed(epoch, path_pic,LQCD["T"],energy, E_true)
        
        

        # Back propagation and optim
        loss.backward()
        opt1.step()
        opt2.step()
        opt3.step()
        schedular1.step()
        schedular2.step()
        schedular3.step()
        lr = schedular1.get_last_lr()[0]

        # check the loss and save the model        
        if loss.item() < Loss_mean:
            checkpoint1 = {
                    'epoch': epoch + 1,
                    'state_dict': Net1.state_dict(),
                    'optimizer': opt1.state_dict(),
                    'loss': loss.item()
                    }
            checkpoint2 = {
                    'epoch': epoch + 1,
                    'state_dict': Net2.state_dict(),
                    'optimizer': opt2.state_dict(),
                    'loss': loss.item()
                    }     
            checkpoint3 = {
                    'epoch': epoch + 1,
                    'state_dict': Net3.state_dict(),
                    'optimizer': opt3.state_dict(),
                    'loss': loss.item()
                    }        
            torch.save(checkpoint1, path_model+"Mud_model.pt")
            torch.save(checkpoint2, path_model+"Ms_model.pt")
            torch.save(checkpoint3, path_model+"Mg_model.pt")
#             print("save model")
            Loss_mean = loss.item()
            

    
    
    toc = time.time()
    trainTime = toc - tic
    print("Training time = ", trainTime)

In [14]:
def Mass_Tmu_test(T, path_model, path_data, device):
    """
    T: Temperature: GeV
    """

    T = torch.FloatTensor(T).reshape(-1,1).to(device)
    T.requires_grad = True


    inp = T
    
    Net1 = Net_Mass(1,1).to(device)
    Net2 = Net_Mass(1,1).to(device)
    Net3 = Net_Mass(1,1).to(device)

    Net1.load_state_dict(torch.load(path_model+"Mud_model.pt",map_location=device)['state_dict'])
    Net2.load_state_dict(torch.load(path_model+"Ms_model.pt",map_location=device)['state_dict'])
    Net3.load_state_dict(torch.load(path_model+"Mg_model.pt",map_location=device)['state_dict'])
    
    Net1.eval()
    Net2.eval()
    Net3.eval()
    # Obtain Mass
    Mud = Net1(inp)
    Ms = Net2(inp)
    Mgluon = Net3(inp)
    coef =  1 / (2 * torch.pi**2)   
    ud_dof = 2 * 2 * 2 * 3  #u,d * q, qbar * spin_dof * color_dof
    s_dof = 2 * 2 * 3
    gluon_dof = 8 * 2       # color dof * polarization dof       

    lnZ_ud =  ud_dof * coef * lnZ_qg(T, Mud, 1) 
    lnZ_s = s_dof * coef * lnZ_qg(T, Ms,1)
    lnZ_gluon = gluon_dof * coef * lnZ_qg(T, Mgluon, -1)

    lnZ_tot = lnZ_ud + lnZ_s + lnZ_gluon
    dlnZdT =  torch.autograd.grad(lnZ_tot, T, grad_outputs=torch.ones_like(T).to(device),create_graph=True)[0]

    pr = T * lnZ_tot  
    ed = T * T * dlnZdT
    dpdT =  torch.autograd.grad(pr, T, grad_outputs=torch.ones_like(T).to(device),create_graph=True)[0]
    dedT =  torch.autograd.grad(ed, T, grad_outputs=torch.ones_like(T).to(device),create_graph=True)[0]
    cs2 = dpdT/dedT
    s_pred = (pr +  ed) / T
    Delta = (ed - 3 * pr)
    mass_loss_1 = torch.where(T >(2.5 * 0.150), torch.abs(Mgluon/Mud - 1.5), torch.tensor([0.]).to(device)) # HTL mass constraint
    mass_loss_2 = torch.where(T >(2.5 * 0.150), torch.abs((Ms  - Mud)/0.09 - 1), torch.tensor([0.]).to(device))
    L_mass =  (mass_loss_1 + mass_loss_2)


    # build the loss function
    loss_mae = nn.L1Loss()


    loss_mc = loss_mae(L_mass , torch.zeros_like(L_mass)).to(device)
    
    np.savez(path_data+'EoS_data_MC', T = T.cpu().detach().numpy(), ed = ed.cpu().detach().numpy(),pr = pr.cpu().detach().numpy(),s_pred = s_pred.cpu().detach().numpy()\
            ,Delta = Delta.cpu().detach().numpy(), loss_mc = loss_mc.cpu().detach().numpy())

In [15]:
# builf the folder
def mkdir(path):

    folder = os.path.exists(path)

    if not folder:                   
        os.makedirs(path)            
        print ("---  new folder...  ---")
        print ("---  OK  ---")

    else:
        print ("---  There is this folder!  ---")
mkdir('data')
mkdir('model')
mkdir('pic')

---  There is this folder!  ---
---  There is this folder!  ---
---  There is this folder!  ---


In [17]:
# parameters
epochs = 5000
path_data = "./data/"
learning_rate = 1e-3
device = torch.device('cpu')

In [18]:
for current_run in range(0,500):
    print("current run:%s"%current_run)
    mkdir("run%s"%current_run)
    mkdir("run%s/data"%current_run)
    mkdir("run%s/model"%current_run)
    mkdir("run%s/pic"%current_run)
    
    test_data = "./run%s/data/"%current_run
    path_model = "./run%s/model/"%current_run
    path_pic = "./run%s/pic/"%current_run
    
    learning_rate = 1e-3
    device = torch.device('cpu')

    Mass_T_train(learning_rate, epochs, path_model, path_pic, path_data, device, current_run)
    df1 = pd.read_csv("./data/hotqcd_1407.6387_err.csv")
    T = df1["T"]
    Mass_Tmu_test(T, path_model, test_data, device)    

current run:0
current run:1
current run:2
current run:3
current run:4
current run:5
current run:6
current run:7
current run:8
current run:9
current run:10
current run:11
current run:12
current run:13
current run:14
current run:15
current run:16
current run:17
current run:18
current run:19
current run:20
current run:21
current run:22
current run:23
current run:24
current run:25
current run:26
current run:27
current run:28
current run:29
current run:30
current run:31
current run:32
current run:33
current run:34
current run:35
current run:36
current run:37
current run:38
current run:39
current run:40
current run:41
current run:42
current run:43
current run:44
current run:45
current run:46
current run:47
current run:48
current run:49
current run:50
current run:51
current run:52
current run:53
current run:54
current run:55
current run:56
current run:57
current run:58
current run:59
current run:60
current run:61
current run:62
current run:63
current run:64
current run:65
current run:66
curre